To open notebook in Colab please click below:
<a href="https://colab.research.google.com/github/bwolfson2/dsclass2022/blob/main/Module_5_KNN/Lecture%205%20-%20KNN-RMS.ipynb" target="_parent"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" /> </a>'

<span style="font-family: Palatino; font-size: 40px; color: purple">
             k-Nearest Neigbors (KNN)
</span>


Spring 2022 - Instructors: Roger Stein and Ben Wolfson -
***

In [ ]:
#If opening in colab run this cell
#!git clone https://github.com/bwolfson2/dsclass2022
#%cd dsclass2022/Module_5_KNN/

In [ ]:
from IPython.core.display import HTML

HTML("""

    <STYLE type="text/css">
       H1 {font-family: Palatino; font-size: 30px; color: purple}
    </STYLE>

""")

KNN algorithms sort all known observations by some measure of “distance” from the observation whose unknown target value we wish to predict.  Distance is calculated using only features (independent variables), _excluding the target_.  Typically the distance is measured as the Euclidian distance (though others are sometimes used).

To make a prediction, we: 
1. sort all of the observations by distance from the unknown observation;
1. create a “neighborhood” of size k (k is a hyper parameter) that includes the k nearest neighbors;
1. aggregate the target values for all of the neighbors (using mean, majority, etc.); and 
1. return aggregated value as the prediction.

Let's see how we would implement KNN from scratch, using only `numpy`.

# Set-up and housekeeping

## Some general imports

In [ ]:
import os
import numpy as np
import pandas as pd
import math

## Define some helper functions that we will use later

# Building KNN from scratch

## Helper functions

### dist_euclidian

In [ ]:
from IPython.display import Image 
Image(url="./images/euclid_dist.png", width=350, height=350)

$$d(A,B)=\sqrt{\Delta_1^2~+~\Delta_2^2 +\dots +\Delta_m^2}.$$

In [ ]:
def dist_euclidian(x1, x2):
    m = len(x1)
    sqr_dists     = [(x1[i]-x2[i])**2 for i in range(m)] 
    sum_sqr_dists = np.sum(sqr_dists)
    d             = np.sqrt(sum_sqr_dists)
    return d

### calc_distances_for_all

In [ ]:
def calc_distances_for_all(x_new,X, dist_metric):
    n     = X.shape[0]
    dists = [dist_metric(x_new,X[i,:]) for i in range(n)]
    return(dists)

## Load data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

#
# Load the Sklearn IRIS dataset
#
iris = datasets.load_iris()
X = iris.data
Y = iris.target
#
# Create train and test split
#
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

In [ ]:
n = X_train.shape[0]
m = X_train.shape[1]
x_test  = X_test[0,:]      # get first test record to demo distance calculations
x_train = X_train[0,:]

## Test our distance function

In [ ]:
difs    = x_train  - x_test
difs_sq = [np.round(difs[i]**2,1) for i in range(m)]

print("x_train:         ", x_train)
print("x_test:          ", x_test)
print("differences:     ", difs)
print("difs^2:          ", np.round(difs_sq,1))
print("sum(difs^2)      ", np.sum(difs_sq))
print("sqrt(sum(difs^2))",np.round(np.sqrt(np.sum(difs_sq)),2))

In [ ]:
d = dist_euclidian(x_train, x_test)
print ("dist_euclidian(x_train, x_test): ", np.round(d,2))

So far, so good.  `dist_euclidian` produces the same results as our hand calculation.  Now let's calculate the distance of our "unknown" observation from all of our training data. 

## Calc distances from all points in test_data

In [ ]:
ds = calc_distances_for_all(x_test,X_train, dist_euclidian)

In [ ]:
np.round(ds[0:5],2)

Now that we have the distance of the "unknown" data point, each point in the training data, we can find the $k$ nearest neighbors to the "unknown" point.

## Find $k$ nearest neighbors

In [ ]:
sorted_idx = np.argsort(ds)

print("Sorted index of `X_train`  from shortest to longest distance:\n")
print(sorted_idx)
print("\n")

print("`X_train` distances sorted from shortest to longest:\n")
print([np.round(ds[i],2) for i in sorted_idx])

## Make prediction (finally!)

In [ ]:
from statistics import mode
from collections import Counter

k = 32
idx = sorted_idx[0:k]
Y_neighbors = Y_train[idx]
print("y neighbors: ", Y_neighbors)
print("\nThe majority class in the neighborhood is: ", mode(Y_neighbors))

counts = Counter(Y_neighbors)
class_labels      = [ l  for l in counts.keys()]
class_frequencies = [ v/k for v in counts.values()]
cf = pd.DataFrame({"class": class_labels,"freq": np.round(class_frequencies,3)})
print("\n", cf)

true_y = Y_test[0]
print("\n** True class ** : ", true_y)

So now that we understand how the mechanics work, let's try to do the same thing, at scale, using `sklearn`.

# Trying out the `iris` dataset using `sklearn`

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load the IRIS dataset

iris = datasets.load_iris()
X = iris.data
y = iris.target

Oops...When we fit the model by hand, we forgot to look at the data first!

Let's remedy that mistake.

## Some basic EDA

In [ ]:
rows, cols = 2, 2
variables  = ['x1',     'x2',   'x3', 'x4']
colors     = ['purple',  'gray',        'green', 'lightblue']
nvars      = len(variables)

fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(7*cols, 7*rows))
axs      = axs.flatten()
params   = {'axes.titlesize':'28', 'xtick.labelsize':'20', 'ytick.labelsize':'20'}
plt.rcParams.update(params)

for i in range(len(variables)):
    v = variables[i]
    axs[i].hist(X_train[:,i], color = colors[i])
    axs[i].set_title(v)
    if i > 0:
        axs[i].sharex(axs[0])   # keep x axis limits the same for easy comp. 
plt.show()    

Notice how the scales of the variables have different magnitudes...

This means that they will make uneven contributions to the distance function since a difference of, say, 1 for a variable that ranges from 0 to 100 probably shouldn't be as 'different' as a difference of 1 for a variable that ranges from 0 to 2.

We can fix this by rescaling the data before we fit the model.

## Preprocessing: Scale the data so each variable has mean  zero and standard deviation 1.

This is often where the modelling can go wrong.  Some Data Scientists accidentally peek into the future when they proprocess the data, because the use the _test_ data to cancluate the preproccing parameters when, by reapplying the _entire_ preprocessing step on the test data, when they should only apply the _transformation_ section, using the parameters that were already estimated from the training data.

Let's take a look at the right way and wrong way to rescale data for KNN.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Variable scaling

std_train = StandardScaler()
std_train.fit(X_train)
X_train_std    = std_train.transform(X_train)
    
              
std_test_ng   = StandardScaler()                # This is wrong...!
std_test_ng.fit(X_test)                         # wrong...!
X_test_std_ng = std_test_ng.transform(X_test)   # wrong...!

X_test_std    = std_train.transform(X_test)     # This is right!

Why is `std_test_ng` wrong?

Let's look at some details about each StandardScalar object we fit.

In [ ]:
print("PARAMETERS OF SCALERS")
print("========== == =======\n")

mu    = std_train.mean_
sigma = np.sqrt(std_train.var_)
print("means, sds from std_train   :\n    mean=", np.round(mu,2),"\n    sd=  ",  np.round(sigma,2))  
print("\n")
mu    = std_test_ng.mean_
sigma = np.sqrt(std_test_ng.var_)
print("means, sds from std_test    :\n    mean=", np.round(mu,2),"\n    sd=  ",  np.round(sigma,2))
print("\n")

print("SUMMARY STATISTICS OF SCALED/UNSCALED DATA")
print("======= ========== == =============== ====\n")
print("TRAINING DATA:")
print("     mean unstandardized:\n",   np.round(pd.DataFrame(X_train).describe(),2))
print("\n     mean standardized:\n",   np.round(pd.DataFrame(X_train_std).describe(),2))


print("\nTEST DATA:")
print("   mean unstandarized:\n",   np.round(pd.DataFrame(X_test).describe().T[['mean','std','50%']].T,2))
print("\n   mean standardized ng (wrong!):\n", np.round(pd.DataFrame(X_test_std_ng).describe().T[['mean','std','50%']].T,2))
print("\n mean standardized (correct):\n", np.round(pd.DataFrame(X_test_std).describe().T[['mean','std','50%']].T,2))

Now we can start building our model...

## Fit the model on the standardized data

To get started, let's try value of $k=32$.

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=32, p=2, weights='uniform', algorithm='auto')
model_knn.fit(X_train_std, Y_train)

## Evaluate the model 

In [ ]:
print('Training accuracy score: %.3f' % model_knn.score(X_train_std, Y_train))
print('Test accuracy score:     %.3f' % model_knn.score(X_test_std, Y_test))

## Finding a better value for $k$

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline


# Create a pipeline
#
pipeline = make_pipeline(StandardScaler(), KNeighborsClassifier())
#
# Create the parameter grid
#
param_grid = [{
    'kneighborsclassifier__n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 32],
    'kneighborsclassifier__p': [1, 2],
    'kneighborsclassifier__weights': ['uniform', 'distance'],
    'kneighborsclassifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
}]
#
# Create a grid search instance
#
gs = GridSearchCV(pipeline, param_grid = param_grid,
                  scoring='accuracy',
                  refit=True,
                  cv=10,
                  verbose=1,
                  n_jobs=2)
#
# Fit the most optimal model
#
gs.fit(X_train, Y_train)
#
# Print the best model parameters and scores
#
print('Best Score: %.3f' % gs.best_score_, '\nBest Parameters: ', gs.best_params_)
#
# Print the model score for test data
#
print('Score: %.3f' % gs.score(X_test, Y_test))


## Fit a model with the 'optimal' value of $k = 5$.

In [ ]:
model_knn_gs = KNeighborsClassifier(n_neighbors=5, p=2, weights='uniform', algorithm='auto')
model_knn_gs.fit(X_train_std, Y_train)

print("                           k=32         k=5")
print(f'Training accuracy score:   {model_knn.score(X_train_std, Y_train):.3f}        {model_knn_gs.score(X_train_std, Y_train):.3f}')
print(f'Test accuracy score:       {model_knn.score(X_test_std, Y_test):.3f} {model_knn_gs.score(X_test_std, Y_test):.3f}')

Y_test_hat =  model_knn_gs.predict(X_test_std)  

**Question: Why did we only use `accuracy_score` rather than `auc` ?**

## Visualize current model predictions

### Set up plot colors

In [ ]:
from icecream import ic
# RGB colors
RED      = (1.00, 0.00, 0.00)
GREEN    = (0.13, 0.55, 0.13)
BLUE     = (0.00, 0.00, 0.93)
LTRED    = (1.00, 0.42, 0.60)
LTBLUE   = (0.28, 0.77, 1.00)
LTGREEN  = (0.29, 0.98, 0.29)

n_train = len(Y_train)
n_test  = len(Y_test)

train_colors = [None] * n_train
for i in range(n_train): 
    if Y_train[i] == 0:
        train_colors[i] = LTRED
    elif Y_train[i] == 1:
        train_colors[i] = LTGREEN        
    else:
        train_colors[i] = LTBLUE
        
test_colors  = [None] * n_test
for i in range(n_test):
    if Y_test_hat[i] == 0:
        test_colors[i] = RED
    elif Y_test_hat[i] == 1:
        test_colors[i] = GREEN        
    else:
        test_colors[i] = BLUE 
  

### Define helper plotting function

In [ ]:
def plot2dKNN(x_train, y_train, x_test, y_test, 
              nrow, ncol, plot_num, xname="x1", yname ="x2", 
              train_colors=(0,0,0), test_colors = (0,0,0)):
    
    ax = plt.subplot(nrow, ncol, plot_num)
    plt.scatter(x_train, y_train,  s=20, c=train_colors)
    plt.scatter(x_test, y_test,  s=70, c=test_colors, marker = "*")
    plt.xlabel(xname, size = 16)
    plt.ylabel(yname, size = 16)
    ax.tick_params(axis='both', which='major', labelsize=12)
    plt.title(xname + ", " + yname + " slice for KNN", size = 22)

### Plot results in 2 dimensions

In [ ]:
               
# ------- Set up plot region and parameters    

plt.figure(figsize=(14, 14))

# ------- row 1

v1_id = 0; v2_id = 1; i = 1
plot2dKNN(X_train_std[:,v1_id], X_train_std[:,v2_id], 
          X_test_std[:,v1_id],  X_test_std[:,v2_id],
          3, 3, i, "x1",  "x2", train_colors, test_colors )   

x1_id = 0; x2_id = 2; i+=1
plot2dKNN(X_train_std[:,v1_id], X_train_std[:,v2_id], 
          X_test_std[:,v1_id],  X_test_std[:,v2_id],
          3, 3, i, "x1",  "x3", train_colors, test_colors )   

x1_id = 0; x2_id = 3; i+=1
plot2dKNN(X_train_std[:,v1_id], X_train_std[:,v2_id], 
          X_test_std[:,v1_id],  X_test_std[:,v2_id],
          3, 3, i, "x1",  "x4", train_colors, test_colors )   

# ------- row 2

i+=1    # spacer to skip unused plot matrix subplot

x1_id = 1; x2_id = 2; i+=1
plot2dKNN(X_train_std[:,v1_id], X_train_std[:,v2_id], 
          X_test_std[:,v1_id],  X_test_std[:,v2_id],
          3, 3, i, "x2",  "x3", train_colors, test_colors )  

x1_id = 1; x2_id = 3; i+=1
plot2dKNN(X_train_std[:,v1_id], X_train_std[:,v2_id], 
          X_test_std[:,v1_id],  X_test_std[:,v2_id],
          3, 3, i, "x2",  "x4", train_colors, test_colors )   

# ------- row 3

i += 1  # spacer to skip unused plot matrix subplot
i += 1  # spacer to skip unused plot matrix subplot

x1_id = 2; x2_id = 3; i+=1
plot2dKNN(X_train_std[:,v1_id], X_train_std[:,v2_id], 
          X_test_std[:,v1_id],  X_test_std[:,v2_id],
          3, 3, i, "x3",  "x4", train_colors, test_colors )  

plt.tight_layout()
plt.show()